In [ ]:

from langchain.chat_models import ChatOpenAI
# 테스트 파일,파워포인트, html, pdfs, images , 등등 로드 할 수 있다.
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
#임베딩 캐시하기
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
#모든 문서를 백터로 바꿨고, 그 백터를 넣은 vectsotre를 활용해서 검색을 해봤다.

cache_dir = LocalFileStore("./.cache/")

cache_embeddings = CacheBackedEmbeddings

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap= 100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
#다음에 우리가 파일을 embedding 할때 첫번째에, 캐시에 embeddings가 있는지 확인
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
#만약없다면 vectors store를 호출할떄 문서들과 함께 openAiembedding를 사용할 것이다.
vectorstore = Chroma.from_documents(docs, embeddings)
#유사도 검색
results = vectorstore.similarity_search("where does winston live")
#위 질문을 하게되면 질문과 관련된 응답을 받게 된다.
results
